# Этап 16: Квантование KV-кеша (Dual-Track)

При генерации текста мы используем **KV-кеш**, чтобы не пересчитывать ключи и значения для предыдущих токенов. В больших моделях этот кеш занимает огромный объем памяти. Согласно разделу 3.2.3 обзора, квантование кеша — ключ к поддержке длинных контекстов.

### Проблема:
Если модель 7B занимает 14 ГБ в FP16, то при контексте 32k токенов её KV-кеш может занять еще столько же. Это удваивает требования к памяти.

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer
from src.model import GPTLanguageModel, device

# 1. Загружаем модели
nanogpt = GPTLanguageModel().to(device)
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
llama = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16).to(device)

def pseudo_quantize_cache(cache_tensor, bits=8):
    # Квантование кеша (обычно по каждому токену)
    # tensor shape: (B, num_heads, seq_len, head_dim)
    q_min, q_max = - (2**(bits-1)), 2**(bits-1) - 1
    
    # Считаем размах для каждого вектора в кеше
    scale = (cache_tensor.abs().max(dim=-1, keepdim=True)[0]) / q_max
    q_x = torch.round(cache_tensor / (scale + 1e-6)).clamp(q_min, q_max)
    
    # Возвращаем «деквантованный» тензор для симуляции потерь
    return q_x * scale

print("Функция квантования кеша готова.")

### nanoGPT: Как бы это выглядело

В nanoGPT мы вручную создаем «кеш» для векторов K и V из одной головы внимания и квантуем его.

In [ ]:
# Симулируем векторы K, созданные в Head.forward (src/model.py:103)
B, T, hs = 1, 256, 64
k_vec = torch.randn(B, T, hs).to(device)

print(f"Размер оригинального K-кеша: {k_vec.nelement() * k_vec.element_size()} байт")

k_quant = pseudo_quantize_cache(k_vec, bits=4)
error = torch.mean((k_vec - k_quant)**2)

print(f"Ошибка после 4-битного квантования кеша: {error.item():.6f}")
print(f"Экономия памяти: 8-кратная (с FP32 до 4-bit)")

### Llama 3: Реальный KV-кеш

В Llama 3 структура кеша сложнее. Мы можем перехватить кеш прямо во время генерации.
Библиотека `transformers` позволяет возвращать `past_key_values`.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
inputs = tokenizer("The quick brown fox", return_tensors="pt").to(device)

with torch.no_grad():
    outputs = llama(**inputs, use_cache=True)
    # past_key_values: кортеж (layer_count) x 2 (K, V) x (B, H, T, hs)
    pkv = outputs.past_key_values
    
    # Возьмем Keys первого слоя
    llama_kv_example = pkv[0][0]
    print(f"Форма KV-кеша Llama (Layers[0], Keys): {llama_kv_example.shape}")
    
    # Симулируем 8-битное квантование всего кеша модели
    llama_kv_quant = pseudo_quantize_cache(llama_kv_example, bits=8)
    
    original_mem = llama_kv_example.nelement() * llama_kv_example.element_size()
    print(f"Память кеша одного слоя (FP16): {original_mem / 1024:.1f} KB")
    print(f"С 8-битным квантованием (INT8): {original_mem / 2048:.1f} KB")

### Итог:
Квантование кеша (особенно до 4 бит) позволяет запускать Llama на устройствах с малым объемом памяти при очень длинных диалогах.